In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [5]:
train_dir = '/content/drive/MyDrive/Bangkit/Capstone/dataset/train'
validation_dir = '/content/drive/MyDrive/Bangkit/Capstone/dataset/valid'

## Check original image dimension

In [7]:
def get_image_dimensions(image_path):
    # Read the image using matplotlib
    img = mpimg.imread(image_path)

    # Get the dimensions from the shape of the NumPy array
    height, width, _ = img.shape

    return width, height

In [8]:
sample_img_paths = ['/content/drive/MyDrive/Bangkit/Capstone/dataset/train/anemic/img_1_120.jpg',
                   '/content/drive/MyDrive/Bangkit/Capstone/dataset/train/anemic/img_1_169.jpg',
                   '/content/drive/MyDrive/Bangkit/Capstone/dataset/train/non-anemic/img_2_36.jpg']

for path in sample_img_paths:
    width, height = get_image_dimensions(path)
    print(width, height)

224 224
224 224
224 224


## List some directories

In [13]:
train_dir = '/content/drive/MyDrive/Bangkit/Capstone/dataset/train'
validation_dir = '/content/drive/MyDrive/Bangkit/Capstone/dataset/valid'

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.9, 1.1],  # Range for random brightness adjustment
    channel_shift_range=10,       # Range for random channel shifts
    fill_mode='nearest'
)

In [10]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

Found 127 images belonging to 2 classes.


In [14]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

Found 56 images belonging to 2 classes.


In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D((3, 3)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 99, 99, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 48, 48, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 15, 15, 64)       

In [34]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [35]:
history = model.fit(
    train_generator,
    epochs=30,  # Use a sufficiently large number of epochs
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/30
4/4 [==============================] - 3s 310ms/step - loss: 0.7173 - accuracy: 0.4488 - val_loss: 0.5430 - val_accuracy: 0.6429
Epoch 2/30
4/4 [==============================] - 2s 460ms/step - loss: 0.4713 - accuracy: 0.8031 - val_loss: 0.4850 - val_accuracy: 0.5714
Epoch 3/30
4/4 [==============================] - 2s 450ms/step - loss: 0.3768 - accuracy: 0.7953 - val_loss: 0.7534 - val_accuracy: 0.8036
Epoch 4/30
4/4 [==============================] - 2s 473ms/step - loss: 0.4471 - accuracy: 0.8425 - val_loss: 0.3911 - val_accuracy: 0.7143
Epoch 5/30
4/4 [==============================] - 3s 788ms/step - loss: 0.3247 - accuracy: 0.8504 - val_loss: 0.3591 - val_accuracy: 0.8393
Epoch 6/30
4/4 [==============================] - 2s 472ms/step - loss: 0.2913 - accuracy: 0.8819 - val_loss: 0.2351 - val_accuracy: 0.8929
Epoch 7/30
4/4 [==============================] - 2s 469ms/step - loss: 0.2605 - accuracy: 0.8976 - val_loss: 0.1610 - val_accuracy: 1.0000
Epoch 8/30
4/4 [====

In [45]:
with open("/content/drive/MyDrive/Bangkit/Capstone/Saved_Model/last_itr.txt", "r") as file:
  current_itr = int(file.readline())
  print("last iteration:", current_itr)

last iteration: 1


In [44]:
current_itr += 1

file_name = f'/content/drive/MyDrive/Bangkit/Capstone/Saved_Model/model_{current_itr}'

model.save(file_name)
print(file_name, "saved")

with open("/content/drive/MyDrive/Bangkit/Capstone/Saved_Model/last_itr.txt", "w") as file:
  file.write(str(current_itr))